In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), ### YOUR CODE HERE
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
  layer.trainable = False ### YOUR CODE HERE
  
# Print the model summary
pre_trained_model.summary()

--2021-12-26 10:01:05--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  50.7MB/s    in 1.7s    

2021-12-26 10:01:07 (50.7 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []   

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7') ### YOUR CODE HERE)
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output ### YOUR CODE HERE

# Expected Output:
# ('last layer output shape: ', (None, 7, 7, 768))

last layer output shape:  (None, 7, 7, 768)


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)### YOUR CODE HERE)(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)### YOUR CODE HERE)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)### YOUR CODE HERE)(x)           

model = Model( pre_trained_model.input, x) ### YOUR CODE HERE, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', ### YOUR CODE HERE,
              metrics = ['accuracy'] ### YOUR CODE HERE,
              )

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [ ]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

# Get the Horse or Human Validation dataset
!gdown --id 1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U
  
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

test_local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(test_local_zip, 'r')
zip_ref.extractall('./training')

val_local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(val_local_zip, 'r')
zip_ref.extractall('./validation')

zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /content/horse-or-human.zip
100% 150M/150M [00:01<00:00, 87.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U
To: /content/validation-horse-or-human.zip
100% 11.5M/11.5M [00:00<00:00, 27.7MB/s]


In [ ]:
train_dir = './training'
validation_dir = './validation'

# Directory with our training horse pictures
train_horses_dir = os.path.join(train_dir, 'horses') ### YOUR CODE HERE
# Directory with our training humans pictures
train_humans_dir = os.path.join(train_dir, 'humans') ### YOUR CODE HERE
# Directory with our validation horse pictures
validation_horses_dir = os.path.join(validation_dir, 'horses') ### YOUR CODE HERE
# Directory with our validation humanas pictures
validation_humans_dir = os.path.join(validation_dir, 'humans') ### YOUR CODE HERE

train_horses_fnames = os.listdir(train_horses_dir) ### YOUR CODE HERE)
train_humans_fnames = os.listdir(train_humans_dir) ### YOUR CODE HERE)
validation_horses_fnames = os.listdir(validation_horses_dir) ### YOUR CODE HERE)
validation_humans_fnames = os.listdir(validation_humans_dir) ### YOUR CODE HERE)

print(len(train_horses_fnames)) ### YOUR CODE HERE)
print(len(train_humans_fnames)) ### YOUR CODE HERE)
print(len(validation_horses_fnames)) ### YOUR CODE HERE)
print(len(validation_humans_fnames)) ### YOUR CODE HERE)

500
527
128
128


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255., ### YOUR CODE HERE)
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. ) ### YOUR CODE HERE)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, ### YOUR CODE HERE)
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir, ### YOUR CODE HERE)
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
callbacks = myCallback() ### YOUR CODE HERE)
history = model.fit( ### YOUR CODE HERE)
            train_generator,
            validation_data = validation_generator,
            epochs = 100,
            validation_steps = 5,
            verbose = 2,
            callbacks=callbacks)

Epoch 1/100
52/52 - 13s - loss: 0.0276 - accuracy: 0.9912 - val_loss: 6.6739e-04 - val_accuracy: 1.0000 - 13s/epoch - 252ms/step
Epoch 2/100
52/52 - 10s - loss: 0.0245 - accuracy: 0.9922 - val_loss: 0.0152 - val_accuracy: 0.9900 - 10s/epoch - 195ms/step
Epoch 3/100
52/52 - 10s - loss: 0.0132 - accuracy: 0.9951 - val_loss: 9.5289e-05 - val_accuracy: 1.0000 - 10s/epoch - 198ms/step
Epoch 4/100
52/52 - 10s - loss: 0.0144 - accuracy: 0.9971 - val_loss: 3.4707e-05 - val_accuracy: 1.0000 - 10s/epoch - 195ms/step
Epoch 5/100
52/52 - 10s - loss: 0.0114 - accuracy: 0.9971 - val_loss: 0.0098 - val_accuracy: 0.9900 - 10s/epoch - 196ms/step
Epoch 6/100
52/52 - 11s - loss: 0.0030 - accuracy: 0.9981 - val_loss: 0.0192 - val_accuracy: 0.9900 - 11s/epoch - 207ms/step
Epoch 7/100

Reached 99.9% accuracy so cancelling training!
52/52 - 10s - loss: 0.0021 - accuracy: 0.9990 - val_loss: 8.9617e-04 - val_accuracy: 1.0000 - 10s/epoch - 196ms/step
